In [11]:
import pandas as pd

total = 0
cities = ["bronx", "brooklyn", "manhattan", "queens", "staten_island"]
for i in cities:
    year = 2022
    while year <= 2024:
        df = pd.read_csv(f"{year}_{i}.csv")
        print("="*60)
        print (f"Data for {i} in {year}:")
        print("="*60)
        print(df.head())
        year += 1
        total +=1

print(total)

# quick scan to see if all xlsx files have been convered to csv correctly, and edits for column names applied smoothly

Data for bronx in 2022:
   BOROUGH NEIGHBORHOOD  BUILDING CLASS CATEGORY TAX CLASS AT PRESENT  BLOCK  \
0        2     BATHGATE  01 ONE FAMILY DWELLINGS                    1   2905   
1        2     BATHGATE  01 ONE FAMILY DWELLINGS                    1   3028   
2        2     BATHGATE  01 ONE FAMILY DWELLINGS                    1   3030   
3        2     BATHGATE  01 ONE FAMILY DWELLINGS                    1   3037   
4        2     BATHGATE  01 ONE FAMILY DWELLINGS                    1   3039   

   LOT  EASE-MENT BUILDING CLASS AT PRESENT                 ADDRESS  \
0   26        NaN                        A9  1667 WASHINGTON AVENUE   
1   24        NaN                        A1   410 EAST 179TH STREET   
2   65        NaN                        A1        4455 PARK AVENUE   
3  101        NaN                        S0     443 EAST 180 STREET   
4   29        NaN                        A5     2327 WASHINGTON AVE   

  APARTMENT NUMBER  ...  RESIDENTIAL UNITS  COMMERCIAL UNITS  TOTAL 

Merging into one large CSV
